## Setup

In [23]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
!./env/bin/pip install -r requirements.txt --quiet

## Step 0: Calibration

#### Modify YOLO Source Code to Disable Suppression

In [3]:
!cd ./env/lib/python3.8/site-packages/ultralytics/yolo/utils && wget https://cq500-mcp.s3.amazonaws.com/ops.py --quiet
print("Updated YOLO source code to disable default suppression!")

Updated YOLO source code to disable default suppression!


#### Run YOLO Inference on Calibration Dataset

In [24]:
# Run Inference on Calibration Dataset
!yolo detect val data="./Step 0: Calibration/calibration.yaml" model="https://cq500-mcp.s3.amazonaws.com/yolo-v8-final-weights.pt" device=cpu seed=829 imgsz=512 project="Step 0: Calibration" name="Calibration Inference Outputs" save_txt=True save_conf=True conf=0 iou=1 max_det=10000 exist_ok=True

# Cleanup
for file in os.listdir('./Step 0: Calibration/Calibration Inference Outputs/'):
    if not os.path.isdir(os.path.join('./Step 0: Calibration/Calibration Inference Outputs/', file)):
        os.system(f'rm -rf "./Step 0: Calibration/Calibration Inference Outputs/{file}"')
os.system("rm -rf ./example_data/calibration/labels.cache")

print(f"Calibration inference complete!")

Found https://cq500-mcp.s3.amazonaws.com/yolo-v8-final-weights.pt locally at yolo-v8-final-weights.pt
Ultralytics YOLOv8.0.123 🚀 Python-3.8.16 torch-2.0.1 CPU
Model summary (fused): 268 layers, 68128383 parameters, 0 gradients
val: Scanning /Users/coopergamble/Desktop/research/mayo/cq500/radiology-mcp-tool
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all          2          2      0.177      0.971      0.586       0.49
                   IPH          2          2      0.177      0.971      0.586       0.49
Speed: 1.0ms preprocess, 684.4ms inference, 0.0ms loss, 1196.4ms postprocess per image
Results saved to Step 0: Calibration/Calibration Inference Outputs
Calibration inference complete!


In [25]:
!./env/bin/python "./Step 0: Calibration/calibrate.py"
print(f"Formatted calibration HNU values!")

100%|█████████████████████████████████████████| 2/2 [00:00<00:00, 135300.13it/s]
Formatted calibration HNU values!


## Step 1: Raw Predictions

In [5]:
# Run Inference on Test Dataset
!yolo detect val data="./Step 1: Raw Predictions/test.yaml" model="https://cq500-mcp.s3.amazonaws.com/yolo-v8-final-weights.pt" device=cpu seed=829 imgsz=512 project="Step 1: Raw Predictions" name="Test Inference Outputs" save_txt=True save_conf=True conf=0 iou=1 max_det=10000 exist_ok=True

# Cleanup
for file in os.listdir('./Step 1: Raw Predictions/Test Inference Outputs/'):
    if not os.path.isdir(os.path.join('./Step 1: Raw Predictions/Test Inference Outputs/', file)):
        os.system(f'rm -rf "./Step 1: Raw Predictions/Test Inference Outputs/{file}"')
os.system("rm -rf ./example_data/test/labels.cache")

print(f"Test inference complete!")

Found https://cq500-mcp.s3.amazonaws.com/yolo-v8-final-weights.pt locally at yolo-v8-final-weights.pt
Ultralytics YOLOv8.0.123 🚀 Python-3.8.16 torch-2.0.1 CPU
Model summary (fused): 268 layers, 68128383 parameters, 0 gradients
val: Scanning /Users/coopergamble/Desktop/research/mayo/cq500/radiology-mcp-tool
val: New cache created: /Users/coopergamble/Desktop/research/mayo/cq500/radiology-mcp-toolkit/example_data/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all          2          2       0.12          1      0.562      0.514
                   IPH          2          2       0.12          1      0.562      0.514
Speed: 1.5ms preprocess, 790.9ms inference, 0.0ms loss, 1297.0ms postprocess per image
Results saved to Step 1: Raw Predictions/Test Inference Outputs
Test inference complete!


In [7]:
!./env/bin/python "./Step 1: Raw Predictions/format_predictions.py"
print(f"Formatted test HNU values!")

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 88.68it/s]


## Step 2: Class-wise NMS

In [10]:
!./env/bin/python "./Step 2: Class-wise NMS/suppress_classwise.py"
print(f"Suppressed non-max test predictions!")

100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1731.75it/s]


## Step 3: Cluster Creation

In [14]:
!./env/bin/python "./Step 3: Cluster Creation/create_clusters.py"
print(f"Created clusters!")

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 21675.99it/s]
Created clusters!


## Step 4 and Step 5: Cluster Filling

In [21]:
!./env/bin/python "./Step 4 and Step 5: Cluster Filling/fill_clusters.py"
print(f"Filled clusters and performed cluster-wise NMS!")

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 28.30it/s]
Filled clusters and performed cluster-wise NMS!


## Step 6: Prediction Sets

In [27]:
!./env/bin/python "./Step 6: Prediction Sets/generate_predictions.py"
print(f"Generated prediction sets!")

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 71697.50it/s]
Generated prediction sets!
